# Introduction

* Thoughts on improve the profitability
> Should find pairs with stronger relationship
>
> Should use some other econometrics model to calculate the spread (?Can we use something other than OLS?)

This notebook aims for providing an experiment of traditional pair trading with parameter autotuning grid search.

This experiment is based on [backtrader](https://www.backtrader.com/) which has an agnosticism philosophy that the data come in as a per-sample basis. Indicators are recalculate when new data come in.

In [1]:
import warnings
import os
import pyfolio
import json
import numpy as np
import pandas as pd
import statsmodels.api as sm
import backtrader as bt

from tqdm import tqdm
from itertools import combinations

from utils.gridsearch import gridsearch
from utils.read2df import read2df
from utils.cointncorr import CointnCorr
from envs.env_gridsearch import KellyCriterionIndicator, PairTrading

warnings.filterwarnings('ignore')

os.makedirs("result/gridsearch", exist_ok=True)

cointncorrtxt = f"result/gridsearch/cointncorr.txt"
os.remove(cointncorrtxt) if os.path.exists(cointncorrtxt) else None

for root, dirs, files in os.walk(f"result/gridsearch/"):
    for file in files:
        os.remove(os.path.join(root, file))

/Users/HSY/miniconda3/envs/uoa-mdt/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


# Data Preparation

Download historical data for `symbols` after `start_date` with selected `freqs` from [`binance-public-data`](https://github.com/binance/binance-public-data/tree/master/python)

We will train data from `start_date` until `trade_date`, and start trade after `trade_date`.

In [2]:
symbols = ['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'XMRUSDT', 'BNBUSDT', 'ADAUSDT', 'DOGEUSDT', 'SOLUSDT', 'TRXUSDT']
start_date = '2022-01-01'
trade_date = '2023-01-01'

# freqs = {'1h':60, '2h':120, '4h':240, '6h':360, '8h':480, '12h':720, '1d':1440}
freqs = {'3m':3, '5m':5, '15m':15, '30m':30}

In [3]:
%%capture
if symbols is None:
    !python binance-public-data/python/download-kline.py \
        -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1
else:
    !python binance-public-data/python/download-kline.py \
        -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1

In [4]:
'''
Read the downloaded OHLCV data into `pandas` dataframe
'''

# dfs = read2df(symbols, freqs)
dfs = read2df(symbols, freqs)

# have a preview
dfs[0].head(10)

,time,open,high,low,close,volume,tic,itvl,datetime
0,1597125779999,0.142880,0.142880,0.142740,0.142870,4.002545e+05,ADAUSDT,3m,2020-08-11 06:02:59.999
1,1597125779999,22.418300,22.418600,22.360000,22.395800,7.928640e+03,BNBUSDT,3m,2020-08-11 06:02:59.999
2,1597125779999,11854.560000,11854.570000,11842.000000,11850.140000,9.037367e+01,BTCUSDT,3m,2020-08-11 06:02:59.999
3,1597125779999,0.003556,0.003559,0.003556,0.003559,4.316000e+05,DOGEUSDT,3m,2020-08-11 06:02:59.999
4,1597125779999,395.100000,395.100000,394.530000,394.950000,6.073353e+02,ETHUSDT,3m,2020-08-11 06:02:59.999
5,1597125779999,58.310000,58.310000,58.210000,58.240000,1.662253e+03,LTCUSDT,3m,2020-08-11 06:02:59.999
6,1597125779999,2.850000,2.850000,2.850000,2.850000,3.600000e+00,SOLUSDT,3m,2020-08-11 06:02:59.999
7,1597125779999,0.021360,0.021360,0.021320,0.021340,2.575011e+06,TRXUSDT,3m,2020-08-11 06:02:59.999
8,1597125779999,94.740000,94.750000,94.670000,94.710000,2.343714e+02,XMRUSDT,3m,2020-08-11 06:02:59.999
9,1597125959999,0.142820,0.142820,0.142600,0.142720,3.272269e+05,ADAUSDT,3m,2020-08-11 06:05:59.999


In [5]:
'''
Set data before `trade_data` as training data, after `trade_data` is trade_data
'''

trains, tests = [], []
for i in range(len(dfs)):
    trains.append(dfs[i][(dfs[i]['datetime'] > start_date) & (dfs[i]['datetime'] < trade_date)].reset_index(drop=True))
    tests.append(dfs[i][dfs[i]['datetime'] >= trade_date].reset_index(drop=True))

# Cointegration and Correlation

We need to make sure that our data is capable for pair trading. 
At least it should have the tendency to merge together therefore we test the coint and corr between every possible two pairs 

Calculate daily coint and corr for all the pairs
Consider 1 day with 1440 minutes.

Delete the pickle file in the `\result` if want to recalculate

In [6]:
'''
Load data from pickle or recalculate
'''

import pickle

# If there is already a pickle file
pickle_file = 'result/cointncorr.pickle'

if os.path.exists(pickle_file):
    with open('result/cointncorr.pickle', 'rb') as pk:
        data = pickle.load(pk)

    freq_position = list(freqs.keys()).index(data[1])

    df0 = dfs[freq_position][dfs[freq_position]['tic']==data[0][0]].reset_index(drop=True)
    df1 = dfs[freq_position][dfs[freq_position]['tic']==data[0][1]].reset_index(drop=True)

    pair, best_freq = data 
    best_pair = f"{pair[0]}_{pair[1]}"

    print("===========================================")
    print(f"Data loaded as: {best_pair} under {best_freq} interval")
    print("===========================================")

else: 
    # If there is no pickle file, then recalculate
    # Takes a looooong time
    tables = CointnCorr(trains, freqs).tabulate()

    with open(cointncorrtxt, "a") as f:
        for k, v in tables.items():
            f.write(f"{k}\n")
            f.write(f"{v}\n\n")
    f.close()

    best_value = 0
    for key in tables.keys():
        for freq in freqs:
            rel = tables[key].at['coint', freq] + tables[key].at['corr', freq]
            if rel > best_value:
                best_value = rel
                best_pair = key
                best_freq = freq

    print("===========================================")
    print(f"Best trading pairs shall be: {best_pair} under {best_freq} interval")
    print(f"the coint is {round(tables[best_pair].at['coint', best_freq]*100, 2)}%")
    print(f"and the corr is {round(tables[best_pair].at['corr', best_freq],3)}")
    print("===========================================")

Data loaded as: BTCUSDT_ETHUSDT under 15m interval


In [7]:
cointncorr = best_pair.split("_"), best_freq

with open('result/cointncorr.pickle', 'wb') as pk:
    pickle.dump(cointncorr, pk)
    
freq_pos = list(freqs.keys()).index(best_freq)

traindata0 = trains[freq_pos][trains[freq_pos]['tic']==best_pair.split('_')[0]].reset_index(drop=True)
traindata1 = trains[freq_pos][trains[freq_pos]['tic']==best_pair.split('_')[1]].reset_index(drop=True)

# How come a datetime of 2022(ish) is converted to 738786.635416655???
datafeed0 = bt.feeds.PandasData(
        dataname=traindata0,
        datetime='datetime',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=None,
    )

datafeed1 = bt.feeds.PandasData(
        dataname=traindata1,
        datetime='datetime',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=None,
    )

datafeeds = [datafeed0, datafeed1]

# Execute the Strategy

We should firstly have a test run on the strategy to make sure it works.

Submitted a PR for Backtrader-OLS results

> https://github.com/mementum/backtrader/pull/487

In [8]:
def cerebro_run(datafeeds, param):
    # Create a Cerebro instance and add the data feed
    cerebro = bt.Cerebro()
    # TODO: should I include `best_pair` as a parameter?
    cerebro.adddata(datafeeds[0], name=best_pair.split('_')[0])
    cerebro.adddata(datafeeds[1], name=best_pair.split('_')[1])

    # Set up other parameters for backtest
    cerebro.broker.set_cash(100000)  # Set initial capital

    # comminfo = PairTradingCommInfo(commission=0.002, margin=1000, mult=10)
    # cerebro.broker.addcommissioninfo(comminfo)

    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturns', compression=60)
    cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
    cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
    # cerebro.addsizer(KellyCriterionSizer)

    cerebro.addstrategy(PairTrading, **param)
    strats = cerebro.run()
    return strats

In [9]:
# A test run
# https://github.com/mementum/backtrader/blob/master/backtrader/indicators/ols.py
# It always returns weird error like the number of params

# 1. The default OLS indicator is precarious, always weird errors like num of params, or sometimes index error
# 2. The custom indicator is precious as well. not trustworthy

param = {'OPEN_THRE':1.5, 'CLOS_THRE':0.1, 'period':10, 'verbose':2, 'prefix':'Experiment'}
experiment = cerebro_run(datafeeds, param)


Open Threshold:1.5, Close Threshold:0.1, period: 10

Starting Value - 100000.00

Ending   Value - 89637.03




# Grid Search the Strategy

The grid search on all the possible results for an optimal outcome

Define `scoring` function and `param_grid` for grid search

then `gridsearch` from `utils.gridsearch`

In [10]:
freq_adjust = freqs[best_freq]

param_grid = {
    'OPEN_THRE': np.arange(2, 10, 2), 
    'CLOS_THRE': np.arange(0.1, 8.1, 2), 
    'period': np.arange(int(100/freq_adjust), int(2000/freq_adjust), int(500/freq_adjust)),
    'verbose': [2],
    'prefix': ['Gridsearch']
}

def scoring(strats):
    score = strats[0].analyzers.Returns.get_analysis()['rtot']
    return score

# included tqdm for progress bar
# Take a loooong time
best_profit, best_params, best_result = gridsearch(cerebro_run, param_grid, scoring, datafeeds)

  2%|▏         | 1/64 [00:49<51:35, 49.14s/it]


Open Threshold:2, Close Threshold:0.1, period: 6

Starting Value - 100000.00

Ending   Value - 80394.57




  3%|▎         | 2/64 [01:36<49:35, 47.98s/it]


Open Threshold:2, Close Threshold:0.1, period: 39

Starting Value - 100000.00

Ending   Value - 98764.21




  5%|▍         | 3/64 [02:23<48:14, 47.46s/it]


Open Threshold:2, Close Threshold:0.1, period: 72

Starting Value - 100000.00

Ending   Value - 107462.71




# Trade with test data
Use the parameter tra

In [ ]:
freq_position = list(freqs.keys()).index(best_freq)

testdata0 = tests[freq_position][tests[freq_position]['tic']==best_pair.split('_')[0]].reset_index(drop=True)
testdata1 = tests[freq_position][tests[freq_position]['tic']==best_pair.split('_')[1]].reset_index(drop=True)

test_datafeed0 = bt.feeds.PandasData(
        dataname=testdata0,
        datetime='datetime',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=None,
    )

test_datafeed1 = bt.feeds.PandasData(
        dataname=testdata1,
        datetime='datetime',
        open='open',
        high='high',
        low='low',
        close='close',
        volume='volume',
        openinterest=None,
)

test_datafeeds = [test_datafeed0, test_datafeed1]

In [ ]:
best_params['verbose']=2
best_params['prefix']= 'Test'
# test_res = cerebro_run(test_datafeeds, best_params)

test_res = cerebro_run(test_datafeeds, best_params)

# Analyze with [Pyfolio](https://pyfolio.ml4trading.io/api-reference.html)

#TODO: We need find a way to calculate the per arbitrage data!... or should we?

Default package has an known error issue out of lack of maintainence, requires to modify the package manually
> https://github.com/quantopian/pyfolio/issues/652

Created a github merge
> https://github.com/quantopian/pyfolio/pull/703

In [ ]:
# res_pyfolio = test_res[0].analyzers.pyfolio.get_analysis()
# res_pyfolio = pd.Series(res_pyfolio['returns'])
# res_pyfolio.index = pd.to_datetime(res_pyfolio.index)
# res_pyfolio = res_pyfolio.astype('float32')
# res_pyfolio

def pyfolio_process(res):
    res_pyfolio = res[0].analyzers.pyfolio.get_analysis()
    res_pyfolio = pd.Series(res_pyfolio['returns'])
    res_pyfolio.index = pd.to_datetime(res_pyfolio.index)
    res_pyfolio = res_pyfolio.astype('float32')
    return res_pyfolio

test_pyfolio = pyfolio_process(test_res)
grid_pyfolio = pyfolio_process(best_result)

In [ ]:
grid_tradeanalyzer = best_result[0].analyzers.tradeanalyzer.get_analysis()
print(json.dumps(grid_tradeanalyzer, indent = 2))

pyfolio.tears.create_full_tear_sheet(returns=pd.Series(grid_pyfolio))


In [ ]:
res_tradeanalyzer = test_res[0].analyzers.tradeanalyzer.get_analysis()
print(json.dumps(res_tradeanalyzer, indent = 2))

pyfolio.tears.create_full_tear_sheet(returns=pd.Series(test_pyfolio))

In [ ]:
import pandas as pd

# Read the text file and split it into individual trades
with open('result/gridsearch/Test_BTCUSDT_ETHUSDT_O35C1P33.txt', 'r') as file:
    trades = file.read().split('---- Close Position @ ')

# Initialize lists to store trade details
dates, buy_currency, sell_currency, buy_price, sell_price, buy_qty, sell_qty = [], [], [], [], [], [], [], []

# Loop through the trades and extract details
for trade in trades:
    trade_details = trade.split('\n')
    
    # Check if the required data exists in this trade block
    if len(trade_details) < 3:
        continue
    
    date = trade_details[0].split('@ ')[1].strip()
    dates.append(date)

    buy_sell_1 = trade_details[1].split()
    buy_currency.append(buy_sell_1[1])
    buy_price.append(float(buy_sell_1[4]))
    buy_qty.append(float(buy_sell_1[8]))

    sell_currency_2 = trade_details[2].split()
    sell_currency.append(sell_currency_2[1])
    sell_price.append(float(sell_currency_2[4]))
    sell_qty.append(float(sell_currency_2[8]))

# Create a DataFrame
data = {
    'Date': dates,
    'Buy Currency': buy_currency,
    'Buy Price': buy_price,
    'Buy Quantity': buy_qty,
    'Sell Currency': sell_currency,
    'Sell Price': sell_price,
    'Sell Quantity': sell_qty
}

df = pd.DataFrame(data)

# Display the DataFrame
print(df)
